In [3]:

#dockerfile test edit
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPU
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from langdetect import detect
from pyspark.sql.functions import col,udf, lit, concat_ws
from pyspark.sql.types import FloatType

from sklearn.metrics import classification_report
import requests
from warcio import ArchiveIterator
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import yfinance as yf
import boto3
import random
import sys 
import numpy as np
import os
#PARAMETERS
path_dl_model = './models/model_dl'
batch_size_max = sys.maxsize -1
num_records_percrawl = 10#int(os.environ['NUMRECORDS']) #number of recors to attempt to extract from each crawl
ticker = 'SPY'

number_warcs_to_analyze = 30 #int(os.environ['NUMWARCS']) #number of warcs to perform sentiment analysis on, goes from most reccent to farther back onse
randomsample = 'n' #str(os.environ['RANSAMPLE']).lower() #Y or N, if Y, then it will take a random sample of warcs to analyze, if N, it will take the most recent warcs

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(['sentence']) \
    .setOutputCol('token')

lemmatizer = Lemmatizer()\
    .setInputCols(['token'])\
    .setOutputCol('lemma')\
  .setDictionary("./sentdat/lemmas_small.txt", key_delimiter="->", value_delimiter="\t")
SentimentDetector = sentiment.SentimentDetector() \
    .setInputCols(['lemma', 'sentence'])\
    .setOutputCol('sentiment_score')\
    .setDictionary('./sentdat/sentiment-big.csv', ',')\

pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    lemmatizer,
    SentimentDetector
])




your 131072x1 screen size is bogus. expect trouble


23/05/04 16:25:29 WARN Utils: Your hostname, DESKTOP-QC3P089 resolves to a loopback address: 127.0.1.1; using 192.168.147.38 instead (on interface eth0)
23/05/04 16:25:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/johan/am111/am111_venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/johan/.ivy2/cache
The jars for the packages stored in: /home/johan/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9a8537dc-d1dc-423c-a5b5-613d21824523;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.4 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.15.0 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failurea

23/05/04 16:25:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('commoncrawl')
warcs = []
for object in my_bucket.objects.filter(Prefix='crawl-data/CC-NEWS/'):
    if object.key.endswith('.warc.gz'):
        warcs.append(object.key)

if randomsample == 'y':
    warcs = random.sample(warcs, number_warcs_to_analyze)
else:
  warcs = warcs[-number_warcs_to_analyze:]

for index, warc in enumerate(warcs):
    warcs[index] = 'https://data.commoncrawl.org/' + warc



In [5]:
path_dl_model = './models/model_dl'
print('starting to load model')
FINDMODEL = PipelineModel.load(path_dl_model)
print('model loaded')


starting to load model


2023-05-04 16:25:52.213187: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


model loaded


In [6]:

def drop_nonfinance_articles(df):
  df = FINDMODEL.transform(df)
  df = df.withColumn('finance', df['financial_model_pred.result'].getItem(0).cast('float'))
  df = df.filter(df['finance'] == 1.0)
  return df


#function to convert time from commoncrawl format to y-m-d
def convert_header_date(date):
    return time.strftime('%Y-%m-%d', time.strptime(date, '%Y-%m-%dT%H:%M:%SZ'))


#obtaining stock data from yahoo finance from 2019 to current date.
currentdate = time.strftime("%Y-%m-%d")
stockdata = yf.download(ticker, start='2010-01-01', end=currentdate)['Adj Close']


[*********************100%***********************]  1 of 1 completed


In [7]:

# creating the main rdd to store the data
#creating scehma to store text and prices
data = StructType([\
  StructField("text", StringType(), True),
    StructField("price", StringType(), True),
    StructField("date", StringType(), True)  
]
)

list_of_rows_batch = []
rows_batch_len = 0
recordsfetched = 0
failures = 0
total = 0 
for warc_url in warcs:
    response = requests.get(warc_url, stream=True)
    if response.ok!=True:
        raise Exception("Error downloading WARC file")
    records = ArchiveIterator(response.raw, arc2warc=True)
    #what this should do is write each record's plaintexxt to a csv file
    for record in records:
        if recordsfetched >= num_records_percrawl:
            total += recordsfetched
            recordsfetched = 0
            print("warc done")
            break
        if record.rec_type == 'response':
            try: 
                html = record.content_stream().read() .decode('utf-8')
                plaintext = BeautifulSoup(html, 'lxml').get_text()
                plaintext = re.sub(r'\s+', ' ', plaintext)
                plaintext = re.sub(r'[^a-zA-Z0-9\s]', '', plaintext).lower()

                #obtains plaintext from the html
                if detect(plaintext) == 'en' and len(plaintext) > 150:  #TODO add classifier here later to ensure its a financial article
                    date = record.rec_headers.get_header('WARC-Date')
                    date = convert_header_date(date)
                    # append the plaintext and price to the batch
                    if date in stockdata.index:
                        list_of_rows_batch.append({'text':plaintext, 'price':float(stockdata[date]), 'date':date})
                        recordsfetched += 1
                        rows_batch_len += 1
                    else:
                        print('date not in stockdata',date)
                        #likely a weekend or holiday, so we will just skip the entire warc
                        break
                else:
                    recordsfetched += 1                          
            except:
                recordsfetched += 1  # because if the entire warc file is not in english or wrong date, we still want to move on to the next one
                failures += 1
                #print("attempt record: ", record.rec_headers.get_header('WARC-Target-URI'), " failed")
                pass

    
df = spark.createDataFrame(list_of_rows_batch, data)
print("size of data: ", total)
print("done")
print("failures: ", failures)


warc done
warc done
warc done
warc done
warc done
warc done
warc done
warc done
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
warc done
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
date not in stockdata 2023-05-04
size of data:  49
done
failures:  1


In [10]:
#sort out the finance articles
df = drop_nonfinance_articles(df)

In [12]:
#INFERENCE
df = pipeline.fit(df).transform(df)

df = df.withColumn("sentiment_score", concat_ws(",", "sentiment_score.result"))

In [14]:
#show the df sentiment_score column
df.select('sentiment_score').show(5, truncate=False)

+---------------+
|sentiment_score|
+---------------+
|positive       |
|positive       |
|positive       |
|negative       |
|positive       |
+---------------+



In [ ]:
#PLOTTING CODE

from pyspark.sql.functions import col, avg
import matplotlib.pyplot as plt

agg_df = df.groupBy("date").agg(avg("sentiment_score").alias("average_sentiment"))

# Convert the aggregated PySpark DataFrame to a Pandas DataFrame
pd_agg_df = agg_df.toPandas()

# Convert 'date' column to pandas datetime object for better plotting
pd_agg_df['date'] = pd.to_datetime(pd_agg_df['date'])

# Sort the DataFrame by date
pd_agg_df = pd_agg_df.sort_values(by="date")

# Plot the average sentiment score as a function of date
plt.plot(pd_agg_df["date"], pd_agg_df["average_sentiment"])
plt.xlabel("Date")
plt.ylabel("Average Sentiment Score")
plt.title("Average Sentiment Score by Date")
plt.show()

# Stop the Spark session
spark.stop()
